# Training

In [1]:
# wandb MA-sae-train: 03ca085d27243e9a7876d511e0c7402d861df34c

In [14]:
from sae_lens import (
    SAETrainingRunner, LanguageModelSAERunnerConfig,
    upload_saes_to_huggingface
)

In [3]:
total_steps        = 75_000
batch_size_tokens  = 4_096
# dataset_path       = "wikitext/wikitext-103-raw-v1"
dataset_path  = "apollo-research/monology-pile-uncopyrighted-tokenizer-EleutherAI-gpt-neox-20b"
log_to_wandb       = True
wandb_project      = "MA-sae-train"
n_checkpoints      = 5
checkpoint_path    = "./checkpoints"

use_autocast       = True
use_autocast_lm    = True
use_compile_sae    = True
use_compile_llm    = True
sae_dtype          = "float16"  # or "bfloat16" if you want to match the model

In [5]:
total_training_tokens = total_steps * batch_size_tokens
lr_warmup_steps       = int(0.10 * total_steps)
lr_decay_steps        = total_steps
l1_warmup_steps       = int(0.20 * total_steps)

1) Using `wikitext-103` for the full run: is its domain coverage sufficient,
   or would a subset of the Pile/OpenWebText better capture diverse contexts? - 
--> !Check
2) Expansion factor=16 on small data can kill features. For a quick run,
   we might try 8 or even 4 to see denser utilization.
3) dtype="float16" + compile_sae can speed up, but have you compared
   reconstruction quality vs float32?

In [6]:
cfg = LanguageModelSAERunnerConfig(
    # — data & model hooks —
    model_name                    = "EleutherAI/pythia-410m-deduped",
    hook_name                     = "blocks.4.hook_mlp_out",
    hook_layer                    = 4,
    d_in                          = 1024,
    dataset_path                  = dataset_path,
    is_dataset_tokenized          = True,#False,
    streaming                     = True,

    # — SAE architecture & sparsity —
    architecture                  = "standard",
    expansion_factor              = 16,
    l1_coefficient                = 2.0,
    l1_warm_up_steps              = l1_warmup_steps,
    normalize_activations         = "expected_average_only_in",
    mse_loss_normalization        = "layer",

    # — init & symmetry —
    b_dec_init_method             = "zeros",
    init_encoder_as_decoder_transpose = True,
    decoder_heuristic_init        = False,

    # — optimization & scheduling —
    lr                            = 5e-5,
    adam_beta1                    = 0.9,
    adam_beta2                    = 0.999,
    lr_scheduler_name             = "cosineannealing",
    lr_warm_up_steps              = lr_warmup_steps,
    lr_decay_steps                = lr_decay_steps,

    # — context & batch sizing —
    context_size                  = 2048, #512,
    train_batch_size_tokens       = batch_size_tokens,

    # — logging, checkpoints & precision —
    training_tokens               = total_training_tokens,
    feature_sampling_window       = 1_000,
    log_to_wandb                  = log_to_wandb,
    wandb_project                 = wandb_project,
    wandb_log_frequency           = 100,
    n_checkpoints                 = n_checkpoints,
    checkpoint_path               = checkpoint_path,
    compile_sae                   = use_compile_sae,
    compile_llm                   = use_compile_llm,
    autocast                      = use_autocast,
    autocast_lm                   = use_autocast_lm,
    device                        = "cuda:0",
    seed                          = 42,
    dtype                         = sae_dtype,
)

In [7]:
print(f"{'TEST_MODE' if TEST_MODE else 'FULL_RUN'} ➞ steps={total_steps}, dataset={dataset_path}")

FULL_RUN ➞ steps=75000, dataset=apollo-research/monology-pile-uncopyrighted-tokenizer-EleutherAI-gpt-neox-20b


In [8]:
# — run training —
sparse_autoencoder = SAETrainingRunner(cfg).run()

Loaded pretrained model EleutherAI/pythia-410m-deduped into HookedTransformer


Resolving data files:   0%|          | 0/338 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/338 [00:00<?, ?it/s]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tilmankerl (tilmankerl-technical-university-of-vienna) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Estimating norm scaling factor:   0%|          | 0/1000 [00:00<?, ?it/s]

Refilling buffer:   0%|          | 0/10 [00:00<?, ?it/s]

Refilling buffer:  10%|█         | 1/10 [00:12<01:49, 12.18s/it]

Refilling buffer:  20%|██        | 2/10 [00:12<00:43,  5.45s/it]

Refilling buffer:  30%|███       | 3/10 [00:13<00:23,  3.30s/it]

Refilling buffer:  40%|████      | 4/10 [00:14<00:13,  2.29s/it]

Refilling buffer:  50%|█████     | 5/10 [00:15<00:08,  1.74s/it]

Refilling buffer:  60%|██████    | 6/10 [00:15<00:05,  1.41s/it]

Refilling buffer:  70%|███████   | 7/10 [00:16<00:03,  1.20s/it]

Refilling buffer:  80%|████████  | 8/10 [00:17<00:02,  1.05s/it]

Refilling buffer:  90%|█████████ | 9/10 [00:18<00:00,  1.05it/s]

Refilling buffer: 100%|██████████| 10/10 [00:18<00:00,  1.12it/s]

                                                                 

Refilling buffer:   0%|          | 0/10 [00:00<?, ?it/s]

Estimating norm scaling factor:   0%|          | 0/1000 [00:19<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 39.50 GiB of which 6.28 GiB is free. Including non-PyTorch memory, this process has 33.21 GiB memory in use. Of the allocated memory 27.81 GiB is allocated by PyTorch, and 4.90 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [17]:
import os
import torch
import torch.distributed as dist

from datasets import load_dataset
from sae_lens import (
    SAETrainingRunner,
    LanguageModelSAERunnerConfig,
    upload_saes_to_huggingface
)

from sae_lens.saes import TopKTrainingSAEConfig
from sae_lens.config import LoggingConfig

# — config parameters —
# projected steps for convergence (gao et al.)
TOTAL_STEPS           = 10 # try different strategies
GLOBAL_BATCH_TOKENS   = 512
TRAIN_TOKENS         = TOTAL_STEPS * GLOBAL_BATCH_TOKENS
LR_WARMUP            = int(0.05 * TOTAL_STEPS)

sae_config = TopKTrainingSAEConfig(
    k                 = 1_024,      # active latents        
    d_sae               = 512 * 8, # # latent dimension (expansion_factor × d_in)
    d_in = 512,
    # expansion_factor  = 8,
)  # no l1_

cfg = LanguageModelSAERunnerConfig(
    # model & hook
    model_name = "HuggingFaceTB/SmolLM2-135M",
    hook_name = "blocks.7.hook_resid_post",
    hook_layer = 7,
                    
    # init_encoder_as_decoder_transpose = True,
    context_size      = 64,

    sae = sae_config,

    # data
    dataset_path = "MisterXY89/lmsys-chat-1m-english-tokenized-smollm-135M",
    is_dataset_tokenized = True,
    streaming = True,

    lr                    = 1e-4,
    adam_beta1            = 0.9,
    adam_beta2            = 0.999,
    lr_scheduler_name     = "cosineannealing",
    lr_warm_up_steps      = LR_WARMUP,
    lr_decay_steps        = TOTAL_STEPS,

    train_batch_size_tokens = GLOBAL_BATCH_TOKENS,
    training_tokens       = TRAIN_TOKENS,


    logger = LoggingConfig(
        log_to_wandb          = True,
        wandb_project         = "SAE-SmolLM-135M",
        wandb_log_frequency   = 100,
    ),
    n_checkpoints         = 5,
    checkpoint_path       = "./checkpoints",

    compile_sae           = False,
    compile_llm           = False,
    autocast              = True,
    autocast_lm           = True,
    dtype                 = "float32",

    device                = f"cuda:0",
    seed                  = 42,

    # use_sparse_kernels    = True,
    # ema_decay             = 0.999,
)
# — data loader —
train_ds = load_dataset(
    cfg.dataset_path,
    split="train",
    streaming=True,
    trust_remote_code=True,
)


sae_model = SAETrainingRunner(cfg, override_dataset=train_ds).run()



Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

You just passed in a dataset which will override the one specified in your configuration: MisterXY89/lmsys-chat-1m-english-tokenized-smollm-135M. As a consequence this run will not be reproducible via configuration alone.


ValueError: HuggingFaceTB/SmolLM2-135M not found. Valid official model names (excl aliases): ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl', 'distilgpt2', 'facebook/opt-125m', 'facebook/opt-1.3b', 'facebook/opt-2.7b', 'facebook/opt-6.7b', 'facebook/opt-13b', 'facebook/opt-30b', 'facebook/opt-66b', 'EleutherAI/gpt-neo-125M', 'EleutherAI/gpt-neo-1.3B', 'EleutherAI/gpt-neo-2.7B', 'EleutherAI/gpt-j-6B', 'EleutherAI/gpt-neox-20b', 'stanford-crfm/alias-gpt2-small-x21', 'stanford-crfm/battlestar-gpt2-small-x49', 'stanford-crfm/caprica-gpt2-small-x81', 'stanford-crfm/darkmatter-gpt2-small-x343', 'stanford-crfm/expanse-gpt2-small-x777', 'stanford-crfm/arwen-gpt2-medium-x21', 'stanford-crfm/beren-gpt2-medium-x49', 'stanford-crfm/celebrimbor-gpt2-medium-x81', 'stanford-crfm/durin-gpt2-medium-x343', 'stanford-crfm/eowyn-gpt2-medium-x777', 'EleutherAI/pythia-14m', 'EleutherAI/pythia-31m', 'EleutherAI/pythia-70m', 'EleutherAI/pythia-160m', 'EleutherAI/pythia-410m', 'EleutherAI/pythia-1b', 'Eleuther